In [1]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import joblib
import sasc.config
import numpy as np
from collections import defaultdict
import pandas as pd
from sasc.modules.fmri_module import convert_module_num_to_voxel_num, add_stability_score
from sasc.config import FMRI_DIR, STORIES_DIR
import re
from copy import deepcopy
from math import ceil
from tqdm import tqdm
def extract_number(string):
    return int(re.findall(r'\d+', string)[0])

# Read all the info from stories into a single pickle file
Note: this session contained 2 different sets of explanations, which we save in separate files.

In [ ]:
# load stuff
# PILOT7 #########################
# output_file = join(sasc.config.RESULTS_DIR,
#                    'processed', "pilot7_story_data.pkl")
# story_mapping = {
#     'roi/uts03___roi_nov30___seed=1_v1': 'GenStory36.npy',
#     'roi/uts03___roi_nov30___seed=2_v1': 'GenStory38.npy',
#     'roi/uts03___roi_nov30___seed=3_v1': 'GenStory40.npy',
# }

# PILOT8 #########################
output_file = join(sasc.config.RESULTS_DIR,
                   'processed', "pilot8_story_data.pkl")
story_mapping = {
    'roi/uts03___roi_nov30___seed=1_v2': 'GenStory37.npy',
    'roi/uts03___roi_nov30___seed=2_v2': 'GenStory39.npy',
    'roi/uts03___roi_nov30___seed=3_v2': 'GenStory41.npy',
}

STORY_DATA_DIR = join(FMRI_DIR, 'brain_tune', 'story_data', '20241204')


# cluster_neighbors = joblib.load(join(FMRI_DIR, "voxel_neighbors_and_pcs", "cluster_neighbors_v1.pkl"))
perfs = joblib.load(join(sasc.config.FMRI_DIR, 'sasc', 'rj_models',
                    'opt_model', 'new_setup_performance.jbl'))

# add keys
stories_data_dict = defaultdict(list)
for story_idx, story_name in enumerate(story_mapping.keys()):
    # add scalar story descriptions
    stories_data_dict["story_name_original"].append(story_name)
    stories_data_dict["story_setting"].append(story_name.split("/")[0])
    stories_data_dict["story_name_new"].append(story_mapping[story_name])
    story_fname = [f for f in os.listdir(join(STORIES_DIR, story_name))
                   if f.startswith('uts03_story')][0]
    story_text = open(join(STORIES_DIR, story_name, story_fname), "r").read()
    stories_data_dict["story_text"].append(story_text)
    # prompts_paragraphs = joblib.load(
    # join(STORIES_DIR, story_name, "prompts_paragraphs.pkl")
    # )
    prompts = open(join(STORIES_DIR, story_name, "prompts.txt"),
                   "r").read().split("\n\n")
    prompts = ['START_PARAGRAPH'] + prompts + ['END_PARAGRAPH']
    prompts_paragraphs = {
        'prompts': prompts,
        'paragraphs': story_text.split("\n\n"),
    }
    assert len(prompts_paragraphs['paragraphs']) == len(
        prompts_paragraphs['prompts']), str(len(prompts_paragraphs['paragraphs'])) + ' ' + str(len(prompts_paragraphs['prompts']))

    # add paragraph-level descriptions
    timings = pd.read_csv(
        # join(STORIES_DIR, story_name, "timings_processed.csv")
        join(STORY_DATA_DIR,
             f'timings_processed{extract_number(story_mapping[story_name])}.csv'),
        header=None,
    ).rename(columns={0: 'word', 2: 'time_running'})
    timings['time_running'] *= 0.75
    # = timings['time_running'].apply(
    # lambda x: x * 0.75)
    # add offset to timings
    # timings['time_running']  # += 10  # -= 5  # += 6
    stories_data_dict["timing"].append(timings)
    stories_data_dict["prompts"].append(prompts_paragraphs["prompts"])
    stories_data_dict["paragraphs"].append(prompts_paragraphs["paragraphs"])

    # add paragraph-level metadata
    rows = pd.read_pickle(
        join(STORIES_DIR, story_name, "rows.pkl"))
    DUMMY_START = pd.DataFrame(
        [{'expl': 'START', 'top_ngrams_module_correct': [],
          'subject': 'UTS03', 'prompt_suffix': ''}])
    DUMMY_END = pd.DataFrame(
        [{'expl': 'END', 'top_ngrams_module_correct': [],
          'subject': 'UTS03', 'prompt_suffix': ''}])
    rows = pd.concat([DUMMY_START, rows, DUMMY_END])
    stories_data_dict["rows"].append(rows)

joblib.dump(stories_data_dict, output_file)

In [ ]:
for k in stories_data_dict:
    print(k, len(stories_data_dict[k][0]))

# resps

In [8]:
pilot_data_dir = join(sasc.config.PILOT_STORY_DATA_DIR, '20241204')

In [ ]:
# load responses
resp_np_files = [stories_data_dict['story_name_new'][i].replace(
    '_resps', '') for i in range(len(stories_data_dict['story_name_new']))]
resps_dict = {
    k: np.load(join(pilot_data_dir, k))
    for k in tqdm(resp_np_files)
}

In [ ]:
for i in range(len(stories_data_dict["story_name_new"])):
    story_name = stories_data_dict["story_name_new"][i]
    timings = stories_data_dict["timing"][i]
    # trs = ceil(timings['time_running'].max() * 0.75 / 2) - 10
    trs = ceil(timings['time_running'].max() / 2) - 10
    print(story_name, trs, resps_dict[story_name].shape[0])

    assert trs == resps_dict[story_name].shape[0]
    # print('resp trs', resps_dict[story_name].shape[0])
    # print(story_name.replace('_resps', ''))